# Web Scraping Part-1:

## Obtain all movies' name and link, write them into a local csv file

In [ ]:
# the package used for this .ipynb file
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def get_all_genres_url():
    # this is a function to obtain all genres and links from a begining url
    all_genres_url = []
    url = 'https://www.imdb.com/chart/moviemeter?ref_=nv_mv_mpm'
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print('something is wrong')
        result = BeautifulSoup(response.content,'lxml')
        genre_list = result.find('ul',{'class':'quicklinks'}).find_all('li')
    except:
        print('oops!')
    
    for genre in genre_list:
        genre_url = 'http://www.imdb.com' + genre.find('a').get('href')
        genre_name = genre.get_text().strip()
        all_genres_url.append((genre_url,genre_name))
    
    return all_genres_url

In [ ]:
# incase of use, we write all genres' names and links into a local csv file
df = pd.DataFrame(feature_movies,columns = ['genre','url'])
df.to_csv('genre_and_link')

In [ ]:
def genre_devided():
    # we read genre_and_link csv and devide all genres into eight vairabls
    # so we can run below code later effeicently
    test = pd.read_csv('genre_and_link',usecols=['genre','url'])
    movies_list = test.values.tolist()
    action = [test['url'][0]]
    adv_ani = test['url'][1:3].tolist()
    bio_com = test['url'][3:5].tolist()
    cri_dra = test['url'][5:8].tolist()
    fam_film = test['url'][8:11].tolist()
    his_mus = test['url'][11:14].tolist()
    musl_rom = test['url'][14:17].tolist()
    sci_wes = test['url'][17:].tolist()
    
    # change return value to obtain differnt genres
    return action

In [ ]:
def get_next_url_for_one_genre(url):
    # this is a function to obtain all pages in one genre by a recursion function
    page_url_list = []
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print('some thing is wrong')
        result = BeautifulSoup(response.content,'lxml')
        try:
            page_url = 'http://www.imdb.com' + result.find('a',{'class':'lister-page-next next-page'}).get('href')
            page_url_list.append(page_url)
            page_url_list.extend(get_next_url_for_one_genre(page_url))
        except:
            return page_url_list
    except:
        print('oops')
        
    return page_url_list

In [ ]:
def get_movie_url_for_one_genre(page_url_list):
    # this is a function to obtain all movies' name and link listed on every page of one genre
    movies_name_link = list()
    for link in page_url_list:
        try:
            response = requests.get(link)
            if not response.status_code == 200:
                return movies_name_link
            result_page = BeautifulSoup(response.content,'lxml')
            movies = result_page.find_all('div',class_= 'lister-item-content')
            try:
                for movie in movies:
                    movie_name = movie.find('a').get_text()
                    movie_link = 'https://www.imdb.com'+movie.find('a').get('href')
                    movies_name_link.append((movie_name,movie_link))
            except:
                movies_name_link.append(())
        
        except:
            return movies_name_link
    return movies_name_link

In [ ]:
def get_all_movies_url(movie_list):
    feature_movies = movie_list
    all_movies_name_link = []
    for link in feature_movies:
        page_url_list = get_all_url_for_one_genre(link)
        movies_name_link = get_movie_url_for_one_genre(page_url_list)
        all_movies_name_link.extend(movies_name_link)
    all_movies_name_link = set(all_movies_name_link)
    return list(all_movies_name_link)

In [ ]:
def obtain_a_csv_file():
    # by running this funtion, we can chose a genre devided to obtain all movies' name and link
    # and write into a csv file
    return_list = get_all_movies_url(genre_devided()) # change which genre devided you want 
                                                      # by chaning the return value of genre_devided()
    df = pd.DataFrame(return_list)
    df.to_csv('action') # change the name of csv_file, match it to the return value of genre_devided()

### Hint: 
* The movie obtained by different genres has duplicate values. We wrote some simple code to remove the duplicate values and get the complete movie name and linked csv file
* file: movie_unique.csv